In [1]:
%config IPCompleter.greedy=True

In [2]:
#adhere to https://github.com/tensorflow/tensorflow/blob/r0.7/tensorflow/examples/tutorials/mnist/input_data.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import csv

data_path = "./weapon_data.csv"

D:\z_outsourced_programs\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def getCsvHeader(filename):
    with open(filename, mode='r') as f:
        reader = csv.reader(f)
        header = next(reader)
        return header
    
def getCsvAsDict(filename):
    header = getCsvHeader(filename)
    result = { h: [] for h in header }
    with open(filename, mode='r') as file:
        for row in csv.DictReader(file):
            for h in header:
                result[h].append(row[h])

    return result

In [4]:
features = getCsvAsDict(data_path)
print(len(features['weapshowname']))
#print(features)

148


In [5]:
def tensorToParameterDict(tensor, cols_to_vars_dict):
    idx =  0 #keep track of the id to find the right value in the processed tensor
    result = dict()
    for key in cols_to_vars_dict:
        '''
        e.g., key = _NumericColumn(key='bdrop', shape=(1,), default_value=None, 
                    dtype=tf.float32, normalizer_fn=None)
        then key[0] = 'bdrop'
        '''
        if key[0] in NUMERICAL_PARAMS:
            result[key[0]] = str(tensor[idx])
        
            '''
            e.g., key = _IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(
                        key='firemode', vocabulary_list=('Automatic', 'Semi-Automatic', 
                        'Single-Action'), dtype=tf.string, default_value=-1, num_oov_buckets=0))
            then key[0] = _VocabularyListCategoricalColumn(
                        key='firemode', vocabulary_list=('Automatic', 'Semi-Automatic', 
                        'Single-Action'), dtype=tf.string, default_value=-1, num_oov_buckets=0)
            and key[0][0] = 'firemode'
            and key[0][0][0] = 'Automatic'
            '''
        elif (len(key[0]) > 0) and key[0][0] in CATEGORICAL_PARAMS:
            for i in range(0, len(key[0][1])-1):
                param = key[0][0] + "_" + key[0][1][i]
                result[param] = str(tensor[idx])
                idx += 1
                
        idx += 1
    return result

In [23]:
#define constants for data processing
CATEGORICAL_PARAMS  = ['type', 'firemode', 'ammo']
NUMERICAL_PARAMS = ['rof', 'damages_first', 'damages_last', 'dmg_distances_first', 'dmg_distances_last', 'initialspeed',
                    'drag', 'magsize', 'shotspershell', 'reloadempty', 'hordispersion', 'verdispersion', 
                    'hiprecoildec', 'hiprecoilright', 'hiprecoilup' ]
WEAPON_TYPES = ['Shotgun', 'Pistol', 'Rifle', 'Submachine Gun', 'Sniper Rifle', 'Light Machine Gun']
WEAPON_FIREMODES = ['Automatic', 'Semi-Automatic', 'Single-Action']

#convert all numerical data to float so they can be used as 'tf.feature_column.numeric_column'
for key, values in features.items():
    if key in NUMERICAL_PARAMS:
        features[key] = [float('{:.4f}'.format(float(value))) for value in values]

#encode categorical data
type_column = tf.feature_column.categorical_column_with_vocabulary_list('type', WEAPON_TYPES)
type_column = tf.feature_column.indicator_column(type_column)

firemode_colum = tf.feature_column.categorical_column_with_vocabulary_list('firemode', WEAPON_FIREMODES)
firemode_colum = tf.feature_column.indicator_column(firemode_colum)

#create list with all feature columns
columns = [tf.feature_column.numeric_column(param) for param in NUMERICAL_PARAMS]
columns.append(type_column)
columns.append(firemode_colum)

#make sure you know whats going on inside
cols_to_vars_dict = {}

#create an input layer with your feature columns and make sure you get the cols_to_vars dictionary!
inputs = tf.feature_column.input_layer(features, columns, cols_to_vars=cols_to_vars_dict, trainable = False)
#print(cols_to_vars_dict)
#print(inputs)

var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer() #needed for the feature columns

weapon_data = [];

#shuffle the whole input layer
tf.set_random_seed(19071991)
with tf.Session() as sess:
    sess.run((var_init, table_init))
    weapon_data = sess.run(tf.random_shuffle(inputs))
    print("Shuffled %i" %len(weapon_data) + " input data")


Shuffled 148 input data


In [42]:
genDict = {}

def addGeneratedWeapon(generatedWeaponTensorDict, dictToAdd):
    for key, value in generatedWeaponTensorDict.items():
        if key not in dictToAdd:
            dictToAdd[key] = []
        dictToAdd[key].append(value)
        
def debug_printDict(dictionary):
    for key, value in dictionary.items():
        print(key, "=", value)

var_init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(var_init)
    
    batch = weapon_data[0:10]
    #print(batch)
    
    
    tensorDict = [tensorToParameterDict(b, cols_to_vars_dict) for b in batch]
    [addGeneratedWeapon(t, genDict) for t in tensorDict]
    #debug_printDict(tensorDict[0])
    
    
debug_printDict(genDict)


damages_first = ['42.0', '26.5', '80.0', '26.5', '80.0', '80.0', '26.5', '34.73', '26.5', '26.5']
damages_last = ['23.0', '15.0', '80.0', '23.0', '80.0', '80.0', '13.5', '15.0', '20.0', '23.0']
dmg_distances_first = ['21.0', '11.0', '0.0', '11.0', '0.0', '0.0', '6.0', '11.0', '11.0', '11.0']
dmg_distances_last = ['60.0', '37.0', '190.0', '35.0', '165.0', '127.5', '31.33', '33.0', '50.0', '35.0']
drag = ['0.0025', '0.0025', '0.002', '0.002', '0.0025', '0.0025', '0.0025', '0.0025', '0.0025', '0.002']
firemode_Automatic = ['0.0', '0.0', '0.0', '1.0', '0.0', '0.0', '0.0', '0.0', '1.0', '1.0']
firemode_Semi-Automatic = ['1.0', '1.0', '0.0', '0.0', '0.0', '0.0', '1.0', '1.0', '0.0', '0.0']
hiprecoildec = ['5.25', '8.0', '3.0', '6.0', '3.0', '4.5', '5.0', '5.0', '6.0', '6.0']
hiprecoilright = ['0.6', '0.36', '1.0', '0.096', '1.0', '1.0', '0.4', '0.1', '0.15', '0.17']
hiprecoilup = ['0.9', '0.7875', '5.0', '0.28', '5.0', '5.0', '1.0', '0.28', '0.2', '0.26']
hordispersion = ['0.0', '0.0', '0.0'